## In-Class JuMP Excercise Option 1: Reservoir Problem

We will try to solve a reservoir optimization Problem using the Julia language and the JuMP Package.
The general problem description is as follows:
One reservoir is being optimized for discharges for power generation and irrigation outflows over a period of 12 months. Inflows of each month are deterministically provided as well as parameters for the different constraints.

First, we will start by installing the appropriate packages for this excercise, this might take a bit of time but is only required to be done once on your machine.

In [ ]:
using Pkg 
Pkg.add("JuMP")
Pkg.add("Clp") #the free solver
Pkg.add("Plots")

Then, we will call those packages from our library to use them...

In [ ]:
using JuMP, Clp, Plots

In [ ]:
#Step 1: Input Data
T=12
inflows = [1200, 1300, 1500, 600, 550, 400, 300, 200, 500, 600, 700, 800]
Irrg_Benefit = [100, 110, 120, 90, 50, 75, 50, 40, 80, 90, 100, 110]
Power_Benefit = [50, 20, 30, 35, 40, 75, 65, 220, 150, 90, 80, 100]
total_flow_max = [1000, 1000, 1100, 900, 900, 750, 650, 700, 800, 900, 1000, 1100]
total_flow_min = [100, 100, 100, 200, 200, 200, 200, 300, 200, 200, 200, 100]
irrg_flow_max = 800
irrg_flow_min = 250
power_flow_max = 750
power_flow_min = 150
storage_max = 2500
storage_min = 250
storage_initial = 1500
water_power_rate = 1.1
#data complete

To declare my Model I give it a model Name (ResvLP) and a Solver (Clp)

In [ ]:
#Declare Model
ResvLP = Model(with_optimizer(Clp.Optimizer))

Decision Variables:
- Outflow for Irrigation
- Outflow for power generation
- Storage

In [ ]:
#variables
@variable(ResvLP, OutflowIrrg[1:T] >= 0) #irrigation outflow

In [ ]:
#Use this space to declare the variables OutflowPower[1:T], Storage[1:T]



Objective Function 

$$\sum_{t=1}^{T} OutflowForIrrigation_t * Irrigation Benefit_t + OutflowPower_t * PowerBenefit_*Rate $$

In [ ]:
#objective #OutflowIrrg[t]*Irrg_Benefit[t] + OutflowPower[t]*Power_Benefit[t]*Water_Power_Rate
@objective(ResvLP, Max, sum(OutflowIrrg[t] * Irrg_Benefit[t] + OutflowPower[t] * Power_Benefit[t]*water_power_rate for t in 1:T))

Constraints:

- Initial Storage:

$$ Storage_t >= StorageInitial $$

In [ ]:
@constraint(ResvLP,Storage[T]>=storage_initial) #end storage

Other Constraints:
- Limits on Irrigation
- Limits on Power transmission
- Limits on Storage
- Mass Balance

In [ ]:
@constraint(ResvLP, IrrgLimit[t in 1:T], irrg_flow_max >= OutflowIrrg[t]>= irrg_flow_min)



In [ ]:
#use this space to define the constraints for Limits on Power flow and Storage:



Mass Balance Constraint:
$$for \\ t \in 1..T\\
\begin{cases}StorageInitial + inflows_t - OutflowIrrigation_t - OutflowPower_t = Storage_t, &\mbox{if } \mbox{t = 1}\\ Storage_{t-1} + inflows_t - OutflowIrrigation_t - OutflowPower_t = Storage_t, &\mbox{if } \mbox{t > 1}\end{cases}$$

Limits on Total Flow
$$TotalFlowMinimum_t <= OutflowIrrigation_t + OutflowPower_t <= TotalFlowMax$$


In [ ]:
#mass balance and Total flow
for t=1:T
    if t==1
        @constraint(ResvLP, storage_initial + inflows[t]-OutflowIrrg[t]-OutflowPower[t] ==Storage[t]);
    else
        @constraint(ResvLP, Storage[t-1] + inflows[t]-OutflowIrrg[t]-OutflowPower[t] ==Storage[t]);
    end
    @constraint(ResvLP,total_flow_min[t]<=OutflowPower[t]+ OutflowIrrg[t]<=total_flow_max[t]);
end

Now that everything is defined, we can solve the problem...

In [ ]:
#Solve
JuMP.optimize!(ResvLP)

In [ ]:
println("RESULTS:
")
println("The total benefit is $(JuMP.objective_value(ResvLP)), The storage at stage
")
	for t in 1:T
			print("$(t) = $(JuMP.value(Storage[t])/t)")
	end

Make nice plots of your results...
Note: The basic Plots package used here can be slow in the first plot, but there are other advanced plotting packages such as Gadfly

In [ ]:
s= JuMP.value.(Storage)
r= JuMP.value.(OutflowIrrg)
Storage_plot= plot(1:12, s, xlabel= "period", ylabel= "m3", title="Variables", label=["Storage Level" ""])

In [ ]:
#With_Irrigation
plot!(1:12, r, label=["Irrigation"]) # ! adds to the existing plot

Other Ways to Contain your Data:

In [ ]:
reservoirs=["RES1", "RES2"] #set of reservoirs
numresv =length(reservoirs)
T=[1,2,3,4,5,6,7,8,9,10,11,12] #12 element array from 1 to 12 representing stages
#inflows( reservoir, T)
inflows = JuMP.Containers.DenseAxisArray([
1200 1300 1500 600 550 400 300 200 500 600 700 850;
1300 1400 1400 650 590 440 320 230 400 500 750 800
], reservoirs, T
)

In [ ]:
inflows["RES2",12]

Farah Rawas